In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cambio = pd.read_excel('../datasets/tc_turistas.xlsx',
                       sheet_name=0)

turista = pd.read_excel('../datasets/tc_turistas.xlsx',
                        sheet_name=2)

# Limpieza de datos

## Cambio

In [3]:
cambio.rename(
    columns={
        'FECHA': 'fecha',
        'DOLAR Oficial': 'dolar_oficial',
        'DOLAR Blue': 'dolar_blue'
    },
    inplace=True
)

Para completar los múltiples datos faltantes de los precios decidimos usar el último valor observado.

Otra forma posible hubise sido usar alguna medida resumen para los valores faltantes, pero en este caso eso haría que fluctúe mucho el incremento del dólar, cuando en realidad tiene una tendencia creciente.

In [4]:
cambio.dolar_oficial.fillna(method='ffill', inplace=True)
cambio.dolar_blue.fillna(method='ffill', inplace=True)

Agrupamos por mes y calculamos el promedio de los valores.

In [5]:
cambio_mensual = cambio.set_index('fecha').resample('M').mean()

In [6]:
cambio_mensual.index = cambio_mensual.index.strftime('%Y-%m')

In [7]:
cambio_mensual.iloc[:, 0:2] = \
    cambio_mensual.iloc[:, 0:2].round(2)

## Turista

In [8]:
turista.rename(
    columns={col: col.lower() for col in turista.columns},
    inplace=True
)

Casteamos las columnas de los turistas llegados de cada país a `int`.

In [9]:
turista[turista.columns[3:]] =\
    turista[turista.columns[3:]].round().astype('int64')

Viendo los máximos de cada campo vemos que hay un valor atípico en uno de ellos.

In [10]:
turista.max()

anio                   2021
mes                     sep
date                  44256
bolivia              187955
brasil               242583
chile       571838802363218
paraguay             138973
uruguay               86375
dtype: object

El valor atípico lo tiene la entrada de año 2018 y mes abril.

In [11]:
turista[turista.chile == 571838802363218]

,anio,mes,date,bolivia,brasil,chile,paraguay,uruguay
63,2018,abr,43191,87508,77917,571838802363218,57205,34822


Decidimos tomar el promedio del año previo y sucesor de 2018 para reemplazar el valor atípico.

In [12]:
turista.loc[turista.chile == 571838802363218, 'chile'] = \
    turista[(turista.anio.isin([2017, 2019])) & (turista.mes == 'abr')]\
        .chile.mean()\
        .astype('int64')

Pasmaos la columna `date` a formato `datetime`.

In [13]:
turista.date = \
    pd.to_datetime(turista.date, unit='d', origin='1899-12-30')

In [14]:
turista['fecha_mensual'] = \
    turista.date.dt.strftime('%Y-%m')

In [15]:
turista_cambio = \
    pd.merge(cambio_mensual, turista, 
            left_index=True, right_on='fecha_mensual', 
            how='left')

In [16]:
turista_cambio = \
    turista_cambio\
        .drop(columns=['anio', 'mes', 'date'])\
        .reindex(columns=['fecha_mensual', 'dolar_oficial',
                        'dolar_blue', 'bolivia', 'brasil',
                        'chile', 'paraguay', 'uruguay'])

In [17]:
turista_cambio.set_index('fecha_mensual')

,dolar_oficial,dolar_blue,bolivia,brasil,chile,paraguay,uruguay
fecha_mensual,,,,,,,
2017-06,16.29,16.50,91354,44950,51902,44106,22150
2017-07,17.38,17.17,139655,67292,90151,74440,25873
2017-08,17.63,18.22,101980,53708,67773,60127,26348
2017-09,17.43,17.95,113370,79189,66417,70245,20975
2017-10,17.66,17.96,108989,86279,74251,65150,27706
2017-11,17.69,18.05,100107,73925,67212,58810,41117
2017-12,17.95,18.31,113631,106259,162180,54932,53165
2018-01,19.27,19.65,156813,124144,98232,47639,72067
2018-02,20.09,20.16,96557,170947,80981,49857,56729


In [18]:
turista_cambio

,fecha_mensual,dolar_oficial,dolar_blue,bolivia,brasil,chile,paraguay,uruguay
53,2017-06,16.29,16.50,91354,44950,51902,44106,22150
54,2017-07,17.38,17.17,139655,67292,90151,74440,25873
55,2017-08,17.63,18.22,101980,53708,67773,60127,26348
56,2017-09,17.43,17.95,113370,79189,66417,70245,20975
57,2017-10,17.66,17.96,108989,86279,74251,65150,27706
58,2017-11,17.69,18.05,100107,73925,67212,58810,41117
59,2017-12,17.95,18.31,113631,106259,162180,54932,53165
60,2018-01,19.27,19.65,156813,124144,98232,47639,72067
61,2018-02,20.09,20.16,96557,170947,80981,49857,56729
62,2018-03,20.48,20.74,77100,87156,83496,66549,54561


### *Preguntar que es la columna date de turista, que hacer con el valor atípico de 2018-abr-chile y por que en el intervalo de cuarenta date no es 0*

# Exploración de datos

# Conclusiones